<a href="https://colab.research.google.com/github/AmazingK2k3/Over_Squashing_GNNs/blob/main/GNN_local_bridges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Using Floyd Warshall Alg - O(V^3)
import numpy as np
import networkx as nx
from collections import Counter

def modified_floyd_warshall(G):
    n = G.number_of_nodes()
    nodes = list(G.nodes())
    node_to_index = {node: i for i, node in enumerate(nodes)}

    d1 = np.full((n, n), np.inf)
    d2 = np.full((n, n), np.inf)
    np.fill_diagonal(d1, 0)

    for u, v, data in G.edges(data=True):
        i, j = node_to_index[u], node_to_index[v]
        weight = data.get('weight', 1)
        d1[i, j] = d1[j, i] = weight
        d2[i, j] = d2[j, i] = weight

    for k in range(n):
        for i in range(n):
            for j in range(n):
                if i != j and j != k and i != k:
                    if d1[i, k] + d1[k, j] < d1[i, j]:
                        d2[i, j] = d1[i, j]
                        d1[i, j] = d1[i, k] + d1[k, j]
                    elif d1[i, k] + d1[k, j] < d2[i, j]:
                        d2[i, j] = d1[i, k] + d1[k, j]

    return d1, d2, nodes

def find_local_bridges(G, d1, d2, nodes):
    n = len(nodes)
    local_bridges = []
    epsilon = 1e-6 if nx.is_weighted(G) else 1

    for i in range(n):
        for j in range(i+1, n):
            if G.has_edge(nodes[i], nodes[j]):
                is_local_bridge = True
                for k in range(n):
                    if k != i and k != j:
                        if d2[i, k] + d2[k, j] <= d1[i, j] + epsilon:
                            is_local_bridge = False
                            break
                if is_local_bridge:
                    local_bridges.append((nodes[i], nodes[j]))

    return local_bridges

def analyze_graph(G):
    print("Graph Information:")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Is weighted: {nx.is_weighted(G)}")

    d1, d2, nodes = modified_floyd_warshall(G)
    local_bridges = find_local_bridges(G, d1, d2, nodes)

    print("\nLocal bridges:")
    for bridge in local_bridges:
        print(f"Edge {bridge}")

    return local_bridges

def analyze_bridge_node_degrees(G, local_bridges):
    bridge_nodes = set([node for bridge in local_bridges for node in bridge])
    bridge_node_degrees = {node: G.degree(node) for node in bridge_nodes}

    print("\nDegrees of nodes involved in local bridges:")
    for node, degree in bridge_node_degrees.items():
        print(f"Node {node}: Degree {degree}")

    degree_counts = Counter(bridge_node_degrees.values())
    print("\nDegree distribution of bridge nodes:")
    for degree, count in sorted(degree_counts.items()):
        print(f"Degree {degree}: {count} node(s)")

    avg_degree = sum(bridge_node_degrees.values()) / len(bridge_node_degrees)
    print(f"\nAverage degree of bridge nodes: {avg_degree:.2f}")

    return bridge_node_degrees

# Example usage
G = nx.Graph()
edges = [(1, 0), (1, 2), (0, 2), (0, 3), (3, 4), (2, 4), (4, 5), (6,7), (4,6), (5,7), (1,8), (3,8), (4,9), (9,0)]
G.add_edges_from(edges)

# Run the analysis
local_bridges = analyze_graph(G)


# Example with a weighted graph
print("\n" + "="*50 + "\n")
G_weighted = nx.Graph()
weighted_edges = [(1, 0, 2), (1, 2, 1), (0, 2, 3), (0, 3, 1), (3, 4, 2), (2, 4, 1), (4, 5, 3), (6, 7, 2), (4, 6, 1), (5, 7, 2), (1, 8, 3), (3, 8, 1), (4, 9, 2), (9, 0, 1)]
G_weighted.add_weighted_edges_from(weighted_edges)

local_bridges_weighted = analyze_graph(G_weighted)


Graph Information:
Number of nodes: 10
Number of edges: 14
Is weighted: False

Local bridges:
Edge (1, 8)
Edge (0, 3)
Edge (0, 9)
Edge (2, 4)
Edge (3, 4)
Edge (3, 8)
Edge (4, 5)
Edge (4, 6)
Edge (4, 9)
Edge (5, 7)
Edge (6, 7)


Graph Information:
Number of nodes: 10
Number of edges: 14
Is weighted: True

Local bridges:
Edge (1, 0)
Edge (1, 2)
Edge (1, 8)
Edge (0, 3)
Edge (0, 9)
Edge (2, 4)
Edge (3, 4)
Edge (3, 8)
Edge (4, 5)
Edge (4, 6)
Edge (4, 9)
Edge (5, 7)
Edge (6, 7)


In [3]:
import networkx as nx
import numpy as np
from collections import defaultdict
from heapq import heappush, heappop

def find_local_bridges(G):
    """
    Find local bridges in a graph using modified Johnson's algorithm.
    A local bridge is an edge that increases the path length between its endpoints when removed.

    Args:
        G (nx.Graph): Input graph

    Returns:
        list: List of tuples (u, v) representing local bridges
    """
    def calculate_shortest_paths(graph, source):
        """Calculate shortest and second shortest paths from source to all vertices."""
        n = graph.number_of_nodes()
        d1 = {v: float('inf') for v in graph.nodes()}  # Primary distances
        d2 = {v: float('inf') for v in graph.nodes()}  # Secondary distances
        d1[source] = 0

        heap = [(0, source)]
        visited = set()

        while heap:
            (dist, u) = heappop(heap)

            if u in visited:
                continue

            visited.add(u)

            for v, data in graph[u].items():
                weight = data.get('weight', 1)
                new_dist = dist + weight

                if new_dist < d1[v]:
                    # Update primary and secondary distances
                    d2[v] = d1[v]
                    d1[v] = new_dist
                    heappush(heap, (new_dist, v))
                elif new_dist < d2[v] and new_dist > d1[v]:
                    # Update only secondary distance
                    d2[v] = new_dist
                    heappush(heap, (new_dist, v))

        return d1, d2

    def is_local_bridge(u, v):
        """Check if edge (u,v) is a local bridge."""
        # Remove edge temporarily
        weight = G[u][v].get('weight', 1)
        G.remove_edge(u, v)

        # Calculate new shortest path length
        try:
            new_shortest_path = nx.shortest_path_length(G, u, v, weight='weight')
        except nx.NetworkXNoPath:
            new_shortest_path = float('inf')

        # Restore edge
        G.add_edge(u, v, weight=weight)

        # Edge is a local bridge if removing it increases the shortest path length
        return new_shortest_path > weight

    def johnson_reweighting():
        """Apply Johnson's reweighting technique to remove negative edges."""
        # Add virtual node connected to all vertices
        H = G.copy()
        virtual = 'virtual'
        for node in G.nodes():
            H.add_edge(virtual, node, weight=0)

        # Run Bellman-Ford from virtual node
        h = nx.bellman_ford_predecessor_and_distance(H, virtual)[1]

        # Remove virtual node
        H.remove_node(virtual)

        # Reweight edges
        for u, v, data in G.edges(data=True):
            weight = data.get('weight', 1)
            G[u][v]['weight'] = weight + h[u] - h[v]

        return h

    # Handle negative edges using Johnson's reweighting
    if any(data.get('weight', 1) < 0 for _, _, data in G.edges(data=True)):
        potential = johnson_reweighting()

    local_bridges = []

    # Check each edge
    for u, v in G.edges():
        # Calculate shortest and second shortest paths
        d1_u, d2_u = calculate_shortest_paths(G, u)

        # Edge is a local bridge if:
        # 1. It's the only shortest path between its endpoints
        # 2. Removing it significantly increases the path length
        if d1_u[v] < d2_u[v] and is_local_bridge(u, v):
            local_bridges.append((u, v))

    return local_bridges

# Example usage:
def test_local_bridges():
    # Create a test graph
    G = nx.Graph()
    edges = [(1, 0), (1, 2), (0, 2), (0, 3), (3, 4), (2, 4), (4, 5), (6,7), (4,6), (5,7), (1,8), (3,8), (4,9), (9,0)]
    G.add_edges_from(edges)

    # Find local bridges
    bridges = find_local_bridges(G)

    return bridges

if __name__ == "__main__":
    bridges = test_local_bridges()
    print("Local bridges found:", bridges)

Local bridges found: [(1, 0), (1, 2), (1, 8), (0, 2), (0, 3), (0, 9), (2, 4), (3, 4), (3, 8), (4, 5), (4, 6), (4, 9), (5, 7), (6, 7)]
